<a href="https://colab.research.google.com/github/chetools/CHE4071_Spring2025/blob/main/FrequencyResponseBodeDiagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift, rhs2TF

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s, w
from sympy import exp, Symbol, simplify
import sympy
import scipy as sp
import scipy.signal as sig

In [3]:
Gc = 3
Gp = 2*exp(-8*s)/(12*s + 1)
G = Gc*Gp

In [4]:
#Evaluate transfer function G over a range of frequencies (ws) after substitution s=jw
#and using lambdify to transform the resulting Sympy expression into a Python function
ws = np.logspace(-2, 0., 100)
Gjw = sympy.lambdify(w, G.subs({s: w*1j}))(ws)
R, I = np.real(Gjw), np.imag(Gjw)
M = np.sqrt(R**2 + I**2)
phi = np.unwrap(np.atan2(I, R)) * 180/np.pi
fig=make_subplots(rows=2,cols=1)
fig.add_scatter(x=ws, y=M, row=1, col=1)
fig.add_scatter(x=ws,y=phi,row=2, col=1)
fig.update_xaxes(type="log", dtick=1, row=1, col=1)
fig.update_yaxes(type="log", row=1, col=1)
fig.update_xaxes(type="log", dtick=1, row=2, col=1)
fig.update_yaxes(dtick=45, row=2, col=1)
fig.update_layout(width=500, height=600, template='plotly_dark', showlegend=False)

In [5]:
Gc = 4.
Gp = 2/(12*s + 1)**3
G = Gc*Gp

In [6]:
Gjw = sympy.lambdify(w, G.subs({s: w*1j}))(ws)
R, I = np.real(Gjw), np.imag(Gjw)
M = np.sqrt(R**2 + I**2)
phi = np.unwrap(np.atan2(I, R)) * 180/np.pi

In [7]:
fig=make_subplots(rows=2,cols=1)
fig.add_scatter(x=ws, y=M, row=1, col=1)
fig.add_scatter(x=ws,y=phi,row=2, col=1)
fig.update_xaxes(type="log", dtick=1, row=1, col=1)
fig.update_yaxes(type="log", row=1, col=1)
fig.update_xaxes(type="log", dtick=1, row=2, col=1)
fig.update_yaxes(dtick=180, row=2, col=1)
fig.update_layout(width=500, height=600, template='plotly_dark', showlegend=False)

In [8]:
Gcl = Gc*Gp/(1+Gc*Gp)
t,y=sim(Gcl, lambda t: 1, N=1000, dt=0.5)

In [9]:
fig=make_subplots(rows=1,cols=1)
fig.add_scatter(x=t, y=y, row=1, col=1)
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [10]:
Ku =4
Pu = 43
Kc = 0.45 * Ku
taui = 0.83*Pu

In [11]:
Gc = Kc*(1 + 1/taui/s)
Gp = 2/(12*s + 1)**3
Gcl = Gc*Gp/(1+Gc*Gp)

In [12]:
t,y=sim(Gcl, lambda t: 1, N=1000, dt=0.5)
fig=make_subplots(rows=1,cols=1)
fig.add_scatter(x=t, y=y, row=1, col=1)
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [25]:
G = 2/(5*s + 1)

In [60]:
Gjw = sympy.lambdify(w, G.subs({s: w*1j}))(np.array([2.]))
R, I = np.real(Gjw), np.imag(Gjw)
M = np.sqrt(R**2 + I**2)
phi = np.unwrap(np.atan2(I, R)) * 180/np.pi

In [32]:
M, phi

(array([0.19900744]), array([-84.28940686]))

In [33]:
t,y=sim(G, lambda t: 1*np.sin(2*t), N= 5000, dt=0.01)
fig=make_subplots(rows=1,cols=1)
fig.add_scatter(x=t, y=y, row=1, col=1)
fig.add_scatter(x=t,y=np.sin(2*t), row=1,col=1)
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [68]:
Gv = 1/(0.1*s+1)
Gci = 1.2
Ginner = Gci*Gv/(1+Gci*Gv)
Gp = 0.12*exp(-5*s)/(12*s+1)
Gco = 1*(1 + 1/(0.5*s))

Gouter = Gco*Ginner*Gp/(1+Gco*Ginner*Gp)


In [69]:
ws = np.logspace(-3, 0., 500)
G = Gco*Ginner*Gp
Gjw = sympy.lambdify(w, G.subs({s: w*1j}))(ws)
R, I = np.real(Gjw), np.imag(Gjw)
M = np.sqrt(R**2 + I**2)
phi = np.unwrap(np.atan2(I, R)) * 180/np.pi
fig=make_subplots(rows=2,cols=1)
fig.add_scatter(x=ws, y=M, row=1, col=1)
fig.add_scatter(x=ws,y=phi,row=2, col=1)
fig.update_xaxes(type="log", dtick=1, row=1, col=1)
fig.update_yaxes(type="log", row=1, col=1)
fig.update_xaxes(type="log", dtick=1, row=2, col=1)
fig.update_yaxes(dtick=180, row=2, col=1)
fig.update_layout(width=500, height=600, template='plotly_dark', showlegend=False)

In [70]:
t,y=sim(Gouter, lambda t: 1, N= 500, dt=1)
fig=make_subplots(rows=1,cols=1)
fig.add_scatter(x=t, y=y, row=1, col=1)
fig.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)